In [78]:
from flask import Flask, request, render_template, url_for, jsonify
import os
import shutil
from data_utils.utils_function import *
from data_utils.get_planet_scenes import search, download_tif_files
import json
import collections

os.environ['CUDA_VISIBLE_DEVICES'] = '0'
collections.Iterable = collections.abc.Iterable
from deploy_model_instances import *
from data_handler import *


class MarineDebrisApp:
    def __init__(self):
        self.app = Flask(__name__)
        self.MAX_CLOUD_PERCENTAGE = 0.5
        self.SEARCH_FOLDER = "searches/2023-05-14_17-21-22.577842"
        self.model = ModelDeployment(model_weights_path="best_model_efficientnetb4_4.h5")
        self.results_list = []
        self.app.route("/")(self.home)
        self.app.route("/Dashboard")(self.dashboard)
        self.app.route('/upload_geojson', methods=['POST'])(self.upload_geojson)
        self.app.route('/get_images')(self.get_images)
        self.app.route('/predict', methods=['POST'])(self.predict)

    def home(self):
        return render_template("index.html", css_file=url_for('static', filename='mapbox-gl-draw.css'),
                               js_file=url_for('static', filename='mapbox-gl-draw.js'))

    def dashboard(self):
        searches_folder_names = os.listdir('searches/')
        return render_template("dashboard.html", searches_folder_names=searches_folder_names)

    def upload_geojson(self):
        posted_file = str(request.files['geojsonFile'].read(), 'utf-8')
        posted_file = json.loads(posted_file)
        geometry_json, start_date_time_formatted, end_date_time_formatted = \
            extract_geometry_timerange_from_json(posted_file)
        print(geometry_json)
        print(start_date_time_formatted)
        print(end_date_time_formatted)
        q_res = search(geometry=geometry_json, start_date=start_date_time_formatted
                     , end_date=end_date_time_formatted, cc=self.MAX_CLOUD_PERCENTAGE)
        self.SEARCH_FOLDER = download_tif_files(q_res)
        print("files were downloaded to {}".format(self.SEARCH_FOLDER))
        return self.dashboard()

    def get_images(self):
        source_dir = os.path.join("static", "predictions").replace("\\", "/")
        target_dir = 'static/images'

        # Copy image files from the source directory to the target directory
        for filename in os.listdir(source_dir):
            source_path = os.path.join(source_dir, filename)
            target_path = os.path.join(target_dir, filename)
            shutil.copy2(source_path, target_path)

        # Get the list of image files in the target directory
        image_files = [f for f in os.listdir(target_dir)
                       if os.path.isfile(os.path.join(target_dir, f))
                       and '.jpg' in f]

        def loads_json(path):
            with open(path, 'r') as file:
                coord = json.load(file)
                return str(coord)

        image_urls = [{
                        'image_url': f'/static/images/{f}',
                        'image_name': f,
                        'image_coordinates':
                            loads_json(
                                f'static/images/{f.replace("prediction","").replace(".jpg",".json")}')
                       }
                         for f in image_files]
        return jsonify(image_urls)

    def predict(self):
        folder = request.form.get('folder')
        print(folder)
        self.SEARCH_FOLDER = os.path.join("searches", folder).replace("\\", "/")
        print(self.SEARCH_FOLDER)
        model_input = data_load(self.SEARCH_FOLDER)
        self.results_list = self.model.execute_job(input_data=model_input)
        predictions_path = os.path.join("static", "predictions").replace("\\", "/")
        writing_results(self.results_list, predictions_path)
        print("predicting")
        self.get_images()
        return self.dashboard()

    def run(self):
        self.app.run()

In [79]:
marine_debris_app = MarineDebrisApp()
marine_debris_app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [16/May/2023 20:47:23] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:47:23] "GET /static/mapbox-gl-draw.js HTTP/1.1" 304 -
127.0.0.1 - - [16/May/2023 20:47:23] "GET /static/mapbox-gl-draw.css HTTP/1.1" 304 -


{'type': 'Polygon', 'coordinates': [[[-86.4898681640625, 16.436085175129968], [-86.4898681640625, 16.43081641246508], [-86.484375, 16.43081641246508], [-86.484375, 16.436085175129968], [-86.4898681640625, 16.436085175129968]]], 'size': 256, 'output_format': 'jpeg'}
2018-03-13T00:00:00Z
2018-03-14T23:00:00Z
no results where been found for the requested time and/or coordinates
creating new dir in /data_utils/searchers/ named current time
call planet api to get query result that given as input
Get Item ID: 
waiting 10 sec before checking activation status again
waiting 10 sec before checking activation status again
waiting 10 sec before checking activation status again
waiting 10 sec before checking activation status again
waiting 10 sec before checking activation status again
waiting 10 sec before checking activation status again
waiting 10 sec before checking activation status again
waiting 10 sec before checking activation status again
waiting 10 sec before checking activation status a

127.0.0.1 - - [16/May/2023 20:50:17] "POST /upload_geojson HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:50:17] "GET /static/loading_animation.gif HTTP/1.1" 304 -


files were downloaded to D:/pythonProjects/final_project/Final_project/app/searches/2023-05-16_20-48-05.553666
2023-05-16_20-48-05.553666
searches/2023-05-16_20-48-05.553666
1/1 [==============================] - 0s 47ms/step


D:\pythonProjects\final_project\Final_project\app\data_handler.py:60: UserWarning: static/predictions\tile_15_11.jpg is a low contrast image
  io.imsave(image_path, result[0])
D:\pythonProjects\final_project\Final_project\app\data_handler.py:61: UserWarning: static/predictions\tile_15_11prediction.jpg is a low contrast image
  io.imsave(prediction_path, result[1])
D:\pythonProjects\final_project\Final_project\app\data_handler.py:60: UserWarning: static/predictions\tile_4_2.jpg is a low contrast image
  io.imsave(image_path, result[0])
D:\pythonProjects\final_project\Final_project\app\data_handler.py:61: UserWarning: static/predictions\tile_4_2prediction.jpg is a low contrast image
  io.imsave(prediction_path, result[1])
D:\pythonProjects\final_project\Final_project\app\data_handler.py:60: UserWarning: static/predictions\tile_27_10.jpg is a low contrast image
  io.imsave(image_path, result[0])
D:\pythonProjects\final_project\Final_project\app\data_handler.py:61: UserWarning: static/pred

predicting


127.0.0.1 - - [16/May/2023 20:53:18] "POST /predict HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /get_images HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /static/images/tile_10_12.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /static/images/tile_11_2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /static/images/tile_10_12prediction.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /static/images/tile_11_2prediction.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /static/images/tile_11_6.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /static/images/tile_11_6prediction.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /static/images/tile_12_2.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /static/images/tile_12_2prediction.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] "GET /static/images/tile_12_3.jpg HTTP/1.1" 200 -
127.0.0.1 - - [16/May/2023 20:53:19] 